# Notebook E-tivity 1 CE4021

Student name: Sean Meade

Student ID: 10128921

<hr style="border:2px solid gray"> </hr>

## Imports

In [ ]:
#None

If you believe required imports are missing, please contact your moderator.

<hr style="border:2px solid gray"> </hr>

## Task

You may add as many cells as you require to complete this task. Refer to the Rubric for E-tivity 1 to see how you will be assessed.

Showing excellent understanding of polynomial derivatives / integrals.

### Understanding Polynomial derivatives / integrals

A polynomial in my own words is a mathematical formula consisting of variables (sometimes called indeterminates) and coefficients. Coefficients are known values and variables are unknown. These variables can have a power associated to it that is a positive integer (for example X^2).

Example of a polynomial: $ y(x) = 3x^3 - 5x^2 + 2x - 1 $

note: a polynomial can have any number of coefficients and variables, the variables can have any order of power that is a positive integer.

#### What are polynomials?

They are equations that can be used to describe linear processes. I always find I understand things when I have a real life example. A great example of a polynomial in the real world is a ball being fired directly up into the air, also known as the "equation for vertical motion". For the sake of this exercise we will fire our ball within a vacuum (i.e. it is not effected by air currents, wind resistance etc). The following formula is a polynomial that represents the vertical height as a function of time ($ y(t) $) with respect to the initial velocity in the y direction ($ V_{iy} $) and time ($ t $). For this example we will consider acceleration due to gravity ($ g $) a constant at 9.8 as we are not shooting our ball into space (unfortunately).

$$ y(t) = V_{iy}t - \frac{1}{2}gt^2 $$

(TODO: Maybe a graph or some visual representation somewhere here)

All this formula is doing is giving you the height above ground (y), at a particular time (t), after being thrown/launched with an initial velocity (v), directly up into the air. For the proper physics nerds they know that g is acceleration due to gravity and is normally taken to 9.81 m/s^2.

For example lets say I launch a ball directly up into the air from a canon (and for simplicity lets say this canon is at ground level). If I know the exact velocity it leaves the ground at I can work out what it's height (y) is above ground for a given time (t).

This is an example of a polynomial in action.

#### So what are derivatives and how do I use them?

Well lets stick with our example. What if I wanted to find the velocity of the ball ($ V_{y} $) for a given time ($ t $)? Well the velocity can be calculated as the change in displacement ($ ds $) divided by the change in time ($ dt $) or $ \frac{ds}{dt} $.

$$ \frac{change~in~displacement}{change~in~time} = \frac{ds}{dt} $$

The derivative is defined as 'an instantaneous rate of change' which, as pointed out in the [video by 3Blue1Brown on the topic](https://youtu.be/9vKqVkMQHKk?si=QGEQitIiNg3zORRg&t=30s), is an oxymoron. "Change is something that happens between separate points in time, and when you blind yourself to all but a single instant, there is no more room for change." For example if you wanted to get the velocity at 3 minutes (at an instant in time) and the velocity requires a change in time to calculate it then it really doesn't make sense at face value.

So like in 3Blue1Brown lets take out the ambiguity and use an actual small value of time for our $ dt $. Let's say these 'instants in time' are actually small time periods of 0.01s. So our change in displacement is the displacement at time $ t + dt $ minus the displacement at time $ t $. Or in terms of of what we know so far (and replacing $ y(t) $ for $ s(t) $):
(TODO: change all references of y(t) to s(t) to avoid confusion)

$$ \frac{ds}{dt} = \frac{s(t + dt) - s(t)}{dt} $$

$$ \frac{ds}{dt} = \frac{(V_{iy}(t + dt) - \frac{1}{2}g(t + dt)^2) - (V_{iy}t - \frac{1}{2}gt^2)}{dt} $$

$$ \frac{ds}{dt} = \frac{V_{iy}(t) + V_{iy}(dt) - \frac{1}{2}gt^2 - \frac{1}{2}g(dt)^2 - g(t)(dt) - V_{iy}(t) + \frac{1}{2}gt^2}{dt} $$

$$ \frac{ds}{dt} = \frac{(V_{iy}(dt) - \frac{1}{2}g(dt)^2 - g(t)(dt)}{dt} $$

$$ \frac{ds}{dt} = V_{iy} - \frac{1}{2}g(dt) - g(t)(dt) $$

Being instantaneous is the same as saying as approaching zero so it is as dt approaches zero ($ {dt \to 0} $)

So the middle term approaches zero.

$$ \frac{ds}{dt} = V_{iy} - g(t)(dt) $$

So with the above equation we can work out the 'instantaneous velocity'.

(TODO: Graph showing time on the x axis where it is marked at t and marked at dt)



### Implementation

Correctly working implementation with efficient use of flow control elements, data types and operators. Clean code with appropriate variable names.  Appropriate documentation of each function.

In [7]:
# TODOs: better naming of variables
# Maybe spread each step out so process is more clear
# Make better TODO list
x = [[1, 2], [3, 4], [7, 8]]

def polynomial_differentiation(input_list, known_var=False):

    output_equation = []

    for a, b in input_list:
        output_equation.append([a * b, b - 1])

    sum = 0
    if known_var:
        for coefficient, power in output_equation:
            sum += (known_var**power)*coefficient

        return sum

    return(output_equation)
        

print(polynomial_differentiation(x))
print(polynomial_differentiation(x, 5))

[[2, 1], [12, 3], [56, 7]]
4376510


### Testing

Test cases cover corner cases without redundancy.

<hr style="border:2px solid gray"> </hr>

## Reflection

Write your reflection in below cell. With reference to the Rubric for E-tivity 1:
- Provide an accurate description of your code with advantages and disadvantages of design choices.
- Compare your approach to alternative (peer) approaches.
- Clearly describe how you have used your peers' work/input and how this has affected your own understanding / insights.

If you have not used peer input, you may state this, but your submission history in Gitlab should clearly show this is the case.